# Import

In [2]:
!pip install optuna

In [3]:
import pandas as pd
import statistics as stats
from random import choice, sample
from functools import partial
from transformers import (AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig,HfArgumentParser,AutoTokenizer,TrainingArguments,Trainer,GenerationConfig)
import os
from openai import OpenAI
import torch
from random import choice, sample
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model, PeftModel, TaskType
import re
import gc
import requests
import json
import re
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import time
from transformers import TrainerCallback
import pynvml
from transformers import set_seed
import time
import math
import pynvml
from transformers import TrainerCallback
import json, time, platform, sys, subprocess, shutil
from pathlib import Path
import torch
from getpass import getpass
import optuna
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import os, re, time, random, statistics as stats
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

try:
    import importlib.metadata as md  # py3.8+
except Exception:
    import importlib_metadata as md   # backport
seed = 42
set_seed(seed)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

#Data generation

In [4]:
# Data generation

# Industry definitions & business descriptions
industries = {
    "tech":       ["AI consultancy", "cloud services", "cybersecurity firm", "autonomous vehicle software", "driver behavior analytics", "fleet optimization service", "smart security system", "predictive energy manager", "network anomaly monitoring"],
    "health":     ["telemedicine platform", "medical imaging diagnostics", "personalized health monitoring", "fitness studio"],
    "finance":    ["fraud detection service", "algorithmic trading platform", "credit scoring startup", "personal budgeting app", "investment advisory service", "cryptocurrency exchange"],
    "ecommerce":  ["product recommendation engine", "dynamic pricing tool", "visual search shopping app", "handcrafted jewelry store", "organic soap store", "pet supplies retailer"],
    "education":  ["adaptive learning system", "automated grading tool", "language learning chatbot", "online tutoring platform", "STEM learning center", "language school"],
    "legal":      ["contract review automation", "legal research assistant", "compliance monitoring tool", "family law firm", "intellectual property practice"],
    "marketing":  ["customer sentiment analyzer", "automated A/B testing tool", "targeted ad optimization platform"],
    "realestate": ["property price predictor", "AI-powered listing recommender", "tenant screening platform", "urban property developer", "luxury real estate broker", "vacation rental manager"],
    "travel":     ["personalized itinerary planner", "flight price prediction engine", "chatbot travel assistant", "boutique travel agency", "adventure tour operator", "luxury cruise planner"],
    "media":      ["music recommendation system", "automated video tagging tool", "game difficulty adaptation engine", "content personalization service"],
    "manufacturing": ["predictive maintenance system", "quality inspection with computer vision", "industrial process optimization"],
    "agriculture":   ["crop yield predictor", "drone-based field analysis", "smart irrigation system"],
    "environment":   ["climate risk modeling service", "wildlife tracking AI", "carbon footprint estimator"],
    "fashion":       ["sustainable clothing brand", "vintage apparel store", "designer footwear line"],
    "cafe":          ["coffee shop", "vegan bakery", "tea lounge"]
}

# Complexity templates (1=short, 2=moderate, 3=rich)
complexity_templates = {
    1: "A {desc} in {city}.",
    2: "A {desc} in {city}, specializing in {specialty}.",
    3: "A {desc} located in {city} that offers {specialty}, targets {audience}, and emphasizes {brand_voice}.",
}

# Context placeholders
cities       = ["Paris", "London", "Berlin", "Tokyo", "New York", "Rome", "Sydney", "Toronto", "Dubai", "Beirut", "Toulouse", "Istanbul", "Los Angelos", "Lille"]
specialties = {
    "tech":         ["cloud migrations", "penetration testing", "AI-driven analytics", "edge computing", "blockchain solutions"],
    "health":       ["virtual consultations", "group fitness programs", "wellness coaching", "remote patient monitoring", "AI-assisted diagnostics"],
    "finance":      ["real-time analytics", "risk assessment", "portfolio management", "fraud detection systems", "automated reporting"],
    "ecommerce":    ["handcrafted designs", "organic ingredients", "pet wellness products", "custom gifts", "personalized recommendations"],
    "education":    ["interactive lessons", "certified tutors", "gamified modules", "adaptive learning paths", "real-time progress tracking"],
    "legal":        ["estate planning", "trademark filings", "corporate compliance", "contract automation", "case law search tools"],
    "realestate":   ["market analysis", "staging services", "holiday rentals", "AI-driven property valuations", "virtual home tours"],
    "travel":       ["custom itineraries", "24/7 support", "off-the-beaten-path tours", "dynamic pricing engines", "language-aware assistants"],
    "fashion":      ["organic fabrics", "artisan craftsmanship", "limited editions", "AI-based style matching", "virtual fitting rooms"],
    "cafe":         ["single-origin pour-overs", "vegan pastries", "latte art classes", "fresh brew", "artisan beans", "organic roasts"],
    "media":        ["automated content tagging", "music personalization", "dynamic video previews", "AI-generated subtitles", "viewer engagement metrics"],
    "manufacturing": ["predictive maintenance", "automated defect detection", "robotic process control", "supply chain forecasting", "real-time quality assurance"],
    "agriculture":  ["precision irrigation", "disease detection via drones", "soil quality monitoring", "yield forecasting", "automated crop spraying"],
    "environment":  ["air quality monitoring", "climate risk modeling", "waste reduction analytics", "renewable energy optimization", "wildlife movement tracking"],
     "marketing": ["customer sentiment analysis", "automated campaign testing", "conversion rate optimization", "real-time engagement tracking", "predictive audience segmentation"
]
}

audiences    = ["millennials", "local foodies", "small businesses", "health-conscious clients", "students", "high-net-worth individuals"]
brand_voices = ["eco-friendly ethos", "luxury experience", "community focus", "tech-savvy vibe", "educational excellence", "financial empowerment"]

# Industry-specific adjectives
adjectives_by_industry = {
    "tech":         ["edge", "quantum", "secure", "smart", "digital", "scalable"],
    "health":       ["well", "fit", "vital", "holistic", "care", "healthy"],
    "finance":      ["secure", "trusted", "wealth", "capital", "insightful", "stable"],
    "ecommerce":    ["handmade", "custom", "organic", "boutique", "eco", "curated"],
    "education":    ["bright", "smart", "engaging", "knowledgeable", "adaptive", "scholarly"],
    "legal":        ["trusted", "prime", "secure", "expert", "legal", "compliant"],
    "realestate":   ["prime", "estate", "lux", "home", "urban", "residence"],
    "travel":       ["wander", "globe", "explore", "journey", "escape", "adventure"],
    "fashion":      ["chic", "vogue", "elegant", "couture", "stylish", "trend"],
    "cafe":         ["fresh", "artisan", "roasted", "barista", "urban", "cozy"],
    "media":        ["viral", "dynamic", "engaging", "immersive", "visual", "creative"],
    "manufacturing":["automated", "precise", "efficient", "industrial", "reliable", "smart"],
    "agriculture":  ["sustainable", "organic", "green", "rural", "fresh", "fertile"],
    "environment":  ["clean", "green", "climate-smart", "eco-friendly", "resilient", "sustainable"],
     "marketing": ["targeted","dynamic","insightful","creative","engaging","data-driven"]
}


# TLD pools (weighted by industry)
TLD_BY_INDUSTRY = {
    "tech":         [".io"]*5 + [".ai"]*5 + [".tech"]*3 + [".com", ".net"],
    "health":       [".health"]*5 + [".fit"]*3 + [".com"],
    "finance":      [".finance", ".invest", ".money", ".com", ".net"],
    "ecommerce":    [".shop"]*5 + [".store"]*5 + [".com", ".net", ".biz"],
    "education":    [".academy", ".edu", ".online", ".co"] + [".com"]*2,
    "legal":        [".law", ".legal", ".com", ".org"],
    "realestate":   [".estate", ".homes", ".property", ".com", ".net"],
    "travel":       [".travel", ".tours", ".vacations", ".com", ".net"],
    "fashion":      [".fashion", ".style", ".boutique", ".com", ".net"],
    "cafe":         [".com", ".net", ".coffee", ".cafe"],
    "media":        [".media", ".tv", ".video", ".studio", ".com"],
    "manufacturing":[ ".industry", ".engineering", ".systems", ".solutions", ".com"],
    "agriculture":  [".farm", ".organic", ".ag", ".green", ".com"],
    "environment":  [".eco", ".green", ".earth", ".solutions", ".org"],
     "marketing" : [".marketing", ".ads", ".media", ".agency", ".com"],
    "default":      [".com", ".net", ".org", ".app", ".co", ".dev", ".info", ".online", ".site"]
}


def sample_tld(industry):
    return choice(TLD_BY_INDUSTRY.get(industry, TLD_BY_INDUSTRY["default"]))

# Domain-pattern functions (now all take industry)
def pattern_hyphen(tokens, city, industry,*kwrags):
    return "-".join(tokens + [city.lower()])

def pattern_concat(tokens, city, industry,*kwrags):
    return "".join(tokens + [city.lower()])

def pattern_get(tokens, city, industry,*kwrags):
    return "-".join(["get"] + tokens + [city.lower()])

def pattern_hq(tokens, city, industry,*kwrags):
    return "".join(tokens) + "-hq"

def pattern_adj_noun_city(tokens, city, industry,*kwrags):
    # pick from this industry's adjectives
    adj = choice(adjectives_by_industry.get(industry, []))
    noun = tokens[0]
    return f"{adj}{noun.capitalize()}{city.lower()}"

DOMAIN_PATTERNS = [
    pattern_hyphen,
    pattern_concat,
    #pattern_get,
    #pattern_hq,
    pattern_adj_noun_city,
]

# “Semantic” pattern: industry-aware adjective + noun + city
def semantic_pattern(tokens, city, industry, spec_phrase):
    """
    Exactly the same as before, but uses the spec_phrase
    from biz_text instead of sampling from all specialties.
    """
    # 1. Take only the first word of the spec you already picked
    spec_first = spec_phrase.split()[0]
    # 2. Build the pool: industry adjectives + that one spec‐word
    pool = adjectives_by_industry.get(industry, []) + [spec_first]
    # 3. Sample your adjective (or that spec‐word)
    adj = choice(pool) if pool else ""
    # 4. Pick a noun‐like token from the description
    noun_tokens = [t for t in tokens if len(t) > 2]
    noun = choice(noun_tokens) if noun_tokens else tokens[0]
    # 5. Return the same concatenation as before
    return f"{adj}{noun}{city.lower()}"


DOMAIN_PATTERNS.append(semantic_pattern)

# 8. Generate exactly one domain per description


def generate_synthetic_data(output_csv_path: str) -> pd.DataFrame:
  rows = []
  for industry, descs in industries.items():
      for desc in descs:
          tokens = desc.lower().split()
          for city in cities:#sample(cities, k=4):           # if you want to sample k cities per desc
              for level, tmpl in complexity_templates.items():
                  spec     = choice(specialties[industry])
                  aud      = choice(audiences)
                  bv       = choice(brand_voices)
                  biz_text = tmpl.format(
                      desc=desc, city=city,
                      specialty=spec, audience=aud, brand_voice=bv
                  )
                  # sample one pattern + TLD
                  pat    = choice(DOMAIN_PATTERNS)
                  name   = pat(tokens, city, industry,spec)
                  tld    = sample_tld(industry)
                  domain = f"{name}{tld}"

                  rows.append({
                      "complexity": level,
                      "industry": industry,
                      "business_description": biz_text,
                      "domain": domain
                  })
  df = pd.DataFrame(rows)
  #ensure_dir(os.path.dirname(output_csv_path))
  df.to_csv(output_csv_path, index=False)
  return df




Preprocessing Data

In [5]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using newline characters
    :param sample: Sample dictionnary
    """

    BUSINESS_KEY = "Business: "
    DOMAIN_KEY = "Domain suggestions: "

    business = f"{BUSINESS_KEY}{sample['business_description']}"
    domain = f"{DOMAIN_KEY}{sample['domain']}"

    parts = [part for part in [business, domain] if part]

    formatted_prompt = "\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [6]:
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """

    inputs = tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
        padding="max_length",#True,
    )
    inputs["labels"] = inputs["input_ids"]

    return inputs
    # Ensure all values are plain lists
    '''return {
        "input_ids": [list(x) for x in inputs["input_ids"]],
        "attention_mask": [list(x) for x in inputs["attention_mask"]],
        "labels": [list(x) for x in inputs["input_ids"]],
    }
'''
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, csv_path):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    raw = load_dataset("csv", data_files=csv_path)["train"]
    #split train/test
    split1 = raw.train_test_split(test_size=0.1, seed=42)
    #split train/eval
    split2 = split1["train"].train_test_split(test_size=0.1, seed=42)

    # Add prompt to each sample
    print("Preprocessing dataset...")
    split = split2.map(create_prompt_formats)#, batched=True)
    test = split1["test"].map(create_prompt_formats)
    # Apply preprocessing to each batch of the dataset & and remove extra fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)

    # each split
    dataset_train = split["train"].map(
        _preprocessing_function,
        batched=True,
        remove_columns=split["train"].column_names,#['complexity', 'industry'],
    )

    dataset_eval = split["test"].map(
        _preprocessing_function,
        batched=True,
        remove_columns=split["test"].column_names,#['complexity', 'industry'],
    )

    dataset_test = test.map(
        _preprocessing_function,
        batched=True,
        remove_columns=test.column_names,#['complexity', 'industry'],
    )
    print(f"Length of training data: {len(dataset_train)}, evaluation data: {len(dataset_eval)}, and test data: {len(dataset_test)}")
    #dataset_train["labels"] = dataset_train["input_ids"]
    #dataset_test["labels"] = dataset_test["input_ids"]

    # Filter out samples that have input_ids exceeding max_length
    #dataset_train = dataset_train.filter(lambda sample: len(sample["input_ids"]) < max_length)
    #dataset_test = dataset_test.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset_train = dataset_train.shuffle(seed=seed)
    dataset_eval = dataset_eval.shuffle(seed=seed)
    dataset_test = dataset_test.shuffle(seed=seed)

    return DatasetDict({
        "train": dataset_train,
        "validation": dataset_eval,
        "test": dataset_test
    })

#Some helper function



In [7]:
# make sure you have: from importlib import metadata as md
# and the usual imports: json, time, platform, subprocess, sys, torch, from pathlib import Path
def _ver(pkg):
    try:
        return md.version(pkg)
    except Exception:
        return None

def write_model_version(trainer, out_dir, version="1.0.0", seed=None, dataset=None,
                        train_metrics=None, eval_metrics=None):

    """
    Write a reproducible manifest for a Hugging Face `Trainer` run.

    Creates `out_dir` (if needed) and saves:
      - `metadata.json`: library versions, hardware (CUDA/cuDNN/GPU), training args,
        model config, dataset fingerprint, and training/eval metrics (incl. full
        `trainer.state.log_history`).
      - `requirements_frozen.txt`: exact Python environment from `pip freeze`.

    Args:
        trainer (transformers.Trainer): Trainer whose `.state`, `.args`, `.model` are inspected.
        out_dir (str | pathlib.Path): Directory to write artifacts to.
        version (str, optional): Model/version tag to record. Default: "1.0.0".
        seed (int | None, optional): Random seed to record.
        dataset (datasets.Dataset | datasets.DatasetDict | None, optional):
            If provided, its (train split) `_fingerprint` is stored.
        train_metrics (dict | None, optional): Training metrics to record; if None,
            a summary is inferred from `trainer.state`.
        eval_metrics (dict | None, optional): Eval metrics to record; if None,
            the most recent `eval_*` entry in `trainer.state.log_history` is used.

    Returns:
        None

    Notes:
        - Requires a `_ver(pkg_name)` helper that returns a package version string.
        - Uses PyTorch to probe CUDA/cuDNN/GPU; values may be None on CPU-only setups.
        - Serialization uses `json.dumps(..., default=str)` to tolerate non-JSON types.
    """

    out = Path(out_dir); out.mkdir(parents=True, exist_ok=True)

    def _last_eval_metrics(state):
        logs = [d for d in state.log_history if any(k.startswith("eval_") for k in d)]
        return logs[-1] if logs else {}

    def _last_train_summary(state):
        # usually the dict that has train_runtime/epoch etc.
        for d in reversed(state.log_history):
            if "train_runtime" in d or "train_loss" in d:
                return d
        return {}

    # minimal + relevant libs
    libs = {
        "python": platform.python_version(),
        "torch": _ver("torch"),
        "transformers": _ver("transformers"),
        "datasets": _ver("datasets"),
        "tokenizers": _ver("tokenizers"),
        "peft": _ver("peft"),
        "accelerate": _ver("accelerate"),
        "bitsandbytes": _ver("bitsandbytes"),
        "numpy": _ver("numpy"),
    }

    # hardware/runtime
    hw = {
        "cuda": getattr(torch.version, "cuda", None),
        "cudnn": torch.backends.cudnn.version() if torch.backends.cudnn.is_available() else None,
        "gpu": torch.cuda.get_device_name(0) if torch.cuda.is_available() else None,
    }

    # dataset fingerprint
    ds_fp = None
    if dataset is not None:
        if hasattr(dataset, "get"):  # DatasetDict-like
            tr = dataset.get("train", None)
            ds_fp = getattr(tr, "_fingerprint", None) if tr is not None else None
        else:
            ds_fp = getattr(dataset, "_fingerprint", None)

    metrics_block = {
        "train": train_metrics or _last_train_summary(trainer.state),
        "eval": eval_metrics or _last_eval_metrics(trainer.state),
        "best_metric": getattr(trainer.state, "best_metric", None),
        "best_model_checkpoint": getattr(trainer.state, "best_model_checkpoint", None),
        "epoch": getattr(trainer.state, "epoch", None),
        "global_step": getattr(trainer.state, "global_step", None),
        "log_history": trainer.state.log_history,
    }

    meta = {
        "version": version,
        "timestamp_utc": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
        "libs": libs,
        "hardware": hw,
        "seed": seed,
        "training_args": trainer.args.to_dict(),
        "metrics": metrics_block,
        "dataset_fingerprint": ds_fp,
        "model_config": getattr(trainer.model, "config", None).to_dict()
                         if hasattr(trainer.model, "config") else None,
    }

    # ensure JSON-serializable
    (out / "metadata.json").write_text(json.dumps(meta, indent=2, default=str))

    # freeze env
    try:
        req = subprocess.check_output([sys.executable, "-m", "pip", "freeze"], text=True)
        (out / "requirements_frozen.txt").write_text(req)
    except Exception:
        pass


In [8]:
def clear_gpu_cache():
    """
    Frees up unused GPU memory.
    Call between large ops to reduce fragmentation and OOM risk.
    """
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [9]:
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
# Function to get the max length of a model
def get_max_length(model):
    """
    Infer the model’s maximum supported sequence length from common config fields.

    Checks, in order, the following attributes on `model.config`:
        - "n_positions"
        - "max_position_embeddings"
        - "seq_length"
    Returns the first truthy value found; otherwise falls back to 1024.

    Args:
        model: A Hugging Face model (or any object with a `.config` exposing the
            attributes above).

    Returns:
        int: The maximum sequence length (tokens).

    Side effects:
        Prints a message indicating the discovered value or that the default (1024) is used.
    """

    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max length: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

In [10]:
def trainable_param_percentage(model: torch.nn.Module) -> float:
    """
    Compute the percentage of model parameters that are trainable.

    Calculates the ratio of parameters with `requires_grad=True` to all parameters
    in the given PyTorch `nn.Module`, expressed as a percentage.

    Args:
        model (torch.nn.Module): The model whose parameters are inspected.

    Returns:
        float: Percentage of trainable parameters in the model (0.0–100.0).
              Returns 0.0 if the model has no parameters.

    """

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    if total_params == 0:
        return 0.0  # Avoid division by zero
    return 100.0 * trainable_params / total_params

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [12]:
# connect to hugging face to use the models
from huggingface_hub import login
token = getpass("Enter your HF token:")
login(token=token)

Enter your HF token:··········


In [13]:
torch.cuda.empty_cache()
clear_gpu_cache()


#Training Functions

##Baseline (full fine-tune) and LoRA training functions

In [30]:
from transformers import TrainerCallback, EarlyStoppingCallback



class EarlyStopNotifier(TrainerCallback):
    def __init__(self):
        self.triggered = False
        self.where = None

    def on_evaluate(self, args, state, control, **kwargs):
        # EarlyStoppingCallback sets this when patience is exceeded
        if control.should_training_stop and not self.triggered:
            self.triggered = True
            self.where = (state.global_step, state.epoch)

    def on_train_end(self, args, state, control, **kwargs):
        if self.triggered and state.is_local_process_zero:
            print(
                f"Early stopping at step {self.where[0]} (epoch {self.where[1]:.2f}). "
                f"Best '{args.metric_for_best_model}' = {state.best_metric} at {state.best_model_checkpoint}."
            )

In [14]:
from datetime import datetime
from transformers import EarlyStoppingCallback
# Baseline
def train_baseline(dataset, tokenizer, output_dir="baseline_full", base_model: str = ""):

    model = AutoModelForCausalLM.from_pretrained(base_model, low_cpu_mem_usage=True)
    model.to(device)
    model.config.pad_token_id = model.config.eos_token_id
    # optional if you want to save to derive
    run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    drive_path = f"/content/drive/MyDrive/FamilyWall/trained_models/run1/{output_dir}_{run_id}"
    path = f"{output_dir}_{run_id}"
    # Optional: keep gradient checkpointing if needed
    model.gradient_checkpointing_enable()

    # 2. Training Arguments
    args = TrainingArguments(
        output_dir=output_dir, # or path for drive saving
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        gradient_accumulation_steps=8,
        num_train_epochs=3,
        learning_rate=5e-4,  # Lower LR typically used for full fine-tuning
        fp16=True,
        logging_steps=3,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        report_to='none',
    )

    # 3. Trainer & Train
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        tokenizer=tokenizer,
    )

    # Print trainable params
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Trainable parameters: {trainable_param_percentage(model):.2f}%")

    trainer.train()
    trainer.save_model(output_dir)
    return tokenizer, model


def train_lora(dataset, tokenizer, output_dir="lora", base_model: str = "", r=16, alpha=32,dropout=0.05,lr=5e-5, max_epochs=3, warm=0, decay=0, extra_msg=None, id=None):

    # Tokenizer & Model
    model = AutoModelForCausalLM.from_pretrained(base_model, low_cpu_mem_usage=True)
    model.to(device)
    model.config.pad_token_id = model.config.eos_token_id
    # enable gradient checkpointing to save memory
    model.gradient_checkpointing_enable()
    # optional if you want to save to derive
    if id is None:
      run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    else:
      run_id = id
    drive_path = f"/content/drive/MyDrive/FamilyWall/models/{output_dir}_{run_id}"
    path = f"{output_dir}_{run_id}"
    # LoRA Configuration
    lora_cfg = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=r,
        lora_alpha=alpha,
        lora_dropout=dropout,
    )
    model = get_peft_model(model, lora_cfg)
    #import pdb; pdb.set_trace() # c for continue q for quit

    # 2.3 Training Arguments
    args = TrainingArguments(
        output_dir=path,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=8,
        num_train_epochs=max_epochs,
        learning_rate=lr,#5e-5,
        weight_decay=decay,
        warmup_ratio=warm,
        fp16=True,
        #logging_steps=50,
        logging_strategy='epoch',
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=5,
        load_best_model_at_end=True,
        report_to = ["tensorboard"],
        label_names=["labels"],
    )

    #print(args)
    # 2.4 Trainer & Train
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        tokenizer=tokenizer,
        callbacks=[PerfLoggingCallback(), EarlyStoppingCallback(early_stopping_patience=3,early_stopping_threshold=0.005), EarlyStopNotifier()],

    )

    print(f"Trainable parameter percentage: {trainable_param_percentage(model):.2f}%")
    train_out = trainer.train()
    train_metrics = train_out.metrics
    eval_metrics = trainer.evaluate()
    print("Saving to:", path)
    trainer.save_model(path)
    trainer.save_model(drive_path)
    write_model_version(trainer, path, version="1.0.0", seed=42, dataset=dataset,
                    train_metrics=train_metrics, eval_metrics=eval_metrics)
    return tokenizer, model

##Hyperparameter optimization

In [15]:
def make_model_init(base_model_dir):
    def model_init(trial=None):
        from peft import LoraConfig, get_peft_model
        gc.collect()
        torch.cuda.empty_cache()
        if trial:
          print("Using hyperparameter trial...")
        else:
          print("Using default LoRA params...")
        model = AutoModelForCausalLM.from_pretrained(
            base_model_dir,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )
        model.config.pad_token_id = model.config.eos_token_id
        model.gradient_checkpointing_enable()
        use_ckpt = trial.suggest_categorical("use_checkpointing", [True, False]) if trial else False
        if use_ckpt:
            model.gradient_checkpointing_enable()

        # Trial-defined LoRA params
        r = trial.suggest_int("lora_r", 4, 64) if trial else 8
        alpha = trial.suggest_int("lora_alpha", 8, 128) if trial else 16
        dropout = trial.suggest_float("lora_dropout", 0.0, 0.3) if trial else 0.05

        lora_cfg = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            inference_mode=False,
            r=r,
            lora_alpha=alpha,
            lora_dropout=dropout,

        )
        model = get_peft_model(model, lora_cfg)
        return model
    return model_init

def hp_space_optuna(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-6, 5e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 15, 30),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.0, 0.3),
    }

def hyperparameter_search(dataset, tokenizer, op_dir, base_model_dir, n_trials=5):
    clear_gpu_cache()
    args = TrainingArguments(
        output_dir=op_dir,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=16,
        fp16=True,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        #logging_steps=50,
        load_best_model_at_end=True,
        report_to="tensorboard",
        label_names=["labels"],
        logging_strategy="epoch",
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    )


    trainer = Trainer(
        model_init=make_model_init(base_model_dir),
        args=args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['validation'],
        tokenizer=tokenizer,
        callbacks=[PerfLoggingCallback(),EarlyStoppingCallback(early_stopping_patience=3,early_stopping_threshold=0.005), EarlyStopNotifier()],
    )



    best = trainer.hyperparameter_search(
        backend="optuna",
        direction="minimize",
        hp_space=hp_space_optuna,
        n_trials=n_trials,
        compute_objective=lambda metrics: metrics["eval_loss"],
    )





    return best, trainer


### Reload best HPO model

In [16]:
from pathlib import Path
import json, re
from transformers import (AutoConfig, AutoTokenizer,
                          AutoModelForCausalLM, AutoModelForSeq2SeqLM)

def _ckpt_step(p: Path):
    m = re.search(r"(\d+)$", p.name)
    return int(m.group(1)) if m else -1

def find_best_checkpoint(run_dir: str):
    """
    Locate the best (or most recent) checkpoint in a Hugging Face training run directory.

    Args:
        run_dir (str): Path to the training run directory.

    Returns:
        str: Filesystem path to the chosen checkpoint (or the run directory if no explicit best).

    Raises:
        FileNotFoundError: If neither a root `trainer_state.json` nor any `checkpoint-*`
            directories are found.

    Notes:
        - Requires a `_ckpt_step(path)` helper that extracts the global step to sort checkpoints.
    """

    run_dir = Path(run_dir)

    # Try root trainer_state.json
    ts = run_dir / "trainer_state.json"
    if ts.exists():
        state = json.loads(ts.read_text())
        return state.get("best_model_checkpoint") or str(run_dir)

    # Look inside checkpoints
    ckpts = sorted(run_dir.glob("checkpoint-*"), key=_ckpt_step)
    if not ckpts:
        raise FileNotFoundError(f"No checkpoints found under {run_dir}")

    # Prefer the newest checkpoint’s trainer_state.json
    for p in reversed(ckpts):
        ts = p / "trainer_state.json"
        if ts.exists():
            state = json.loads(ts.read_text())
            return state.get("best_model_checkpoint") or str(p)

    # Fallback: use the last checkpoint even if no state file
    return str(ckpts[-1])

def load_model_from_run(run_dir: str):
    """
    Load a PEFT/LoRA model from a training run directory.

    Args:
        run_dir (str): Path to the training run directory containing checkpoints
            (e.g., `checkpoint-*`) and/or `trainer_state.json`.

    Returns:
        peft.PeftModel: The base `AutoModelForCausalLM` wrapped with the loaded adapters.
    """

    adapter_dir = find_best_checkpoint(run_dir)
    cfg = PeftConfig.from_pretrained(adapter_dir)
    base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
    model = PeftModel.from_pretrained(base, adapter_dir)
    return model



## Qwen training

In [14]:
def _pick_targets(model):
    # Find which common projection names this checkpoint actually uses
    candidates = ["q_proj","k_proj","v_proj","o_proj",
                  "gate_proj","up_proj","down_proj",
                  "W_pack","qkv_proj"]
    present = [c for c in candidates if any(c in n for n, _ in model.named_modules())]
    return present

def train_qwen_lora(dataset, tokenizer, output_dir="qwen", base_model="Qwen/Qwen2.5-7B",
                    r=16, alpha=32, dropout=0.05, lr=5e-5, max_epochs=3, warm=0, decay=0,
                    extra_msg=None, id=None, use_qlora=True):

    # paths
    run_id = id or time.strftime("%Y-%m-%d_%H-%M-%S")
    path = f"{output_dir}_{run_id}"
    drive_path = f"/content/drive/MyDrive/FamilyWall/models/{path}"

    # tokenizer
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.padding_side = "right"

    # load model
    quant_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    ) if use_qlora else None

    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        low_cpu_mem_usage=True,
        device_map="auto",
        torch_dtype=torch.bfloat16 if use_qlora else None,
        quantization_config=quant_cfg,
        trust_remote_code=True,
    )
    model.config.pad_token_id = model.config.eos_token_id
    model.config.use_cache = False
    model.gradient_checkpointing_enable()
    if use_qlora:
        model = prepare_model_for_kbit_training(model)

    # LoRA
    targets = _pick_targets(model)
    if not targets:
        raise ValueError("Could not find any target modules for LoRA. Inspect model.named_modules().")
    lora_cfg = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=r, lora_alpha=alpha, lora_dropout=dropout,
        target_modules=targets,
    )
    model = get_peft_model(model, lora_cfg)

    # quick sanity check
    model.print_trainable_parameters()
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    if trainable == 0:
        raise RuntimeError("LoRA attached 0 trainable params. Check target_modules.")

    # data collator
    collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    # training args
    args = TrainingArguments(
        output_dir=path,
        per_device_train_batch_size=4 if use_qlora else 1,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=16,
        num_train_epochs=max_epochs,
        learning_rate=lr,
        weight_decay=decay,
        warmup_ratio=warm,
        bf16=True,
        logging_strategy="epoch",
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=5,
        load_best_model_at_end=True,
        report_to=["tensorboard"],
        label_names=["labels"],
    )

    # --- callbacks (guard NVML) ---
    cbs = [EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.005), EarlyStopNotifier()]
    try:
        import pynvml
        pynvml.nvmlInit()
        cbs.insert(0, PerfLoggingCallback())
    except Exception:
        print("NVML not available; skipping PerfLoggingCallback.")

    # trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        tokenizer=tokenizer,
        data_collator=collator,
        callbacks=cbs,
    )

    print(f"Trainable parameter percentage: {100.0 * trainable / sum(p.numel() for p in model.parameters()):.2f}%")
    train_out = trainer.train()
    train_metrics = train_out.metrics
    eval_metrics = trainer.evaluate()

    print("Saving to:", path)
    trainer.save_model(path)
    try:
        trainer.save_model(drive_path)
    except Exception as e:
        print(f"Drive save skipped: {e}")

    write_model_version(trainer, path, version="1.0.0", seed=42, dataset=dataset,
                        train_metrics=train_metrics, eval_metrics=eval_metrics)
    return tokenizer, model



#Functions to test the models

## Suggest Function

In [15]:
# 3. Define a generic suggestion helper to generate domains using the trained models
def suggest_domains(model, tokenizer, biz_desc, n=3):
    """
    Generate domain name ideas from a business description using a HF text model.

    Builds a simple prompt:
        "Business: {biz_desc}\nDomain suggestions:"
    Tokenizes it on the model's device, switches the model to eval mode, and
    samples `n` continuations with `generate()` (temperature=0.7, max_new_tokens=32).
    Returns only the newly generated text after the prompt for each sequence.

    Args:
        model: A Hugging Face causal LM.
        tokenizer: Matching tokenizer for `model`.
        biz_desc (str): Short description of the business.
        n (int, optional): Number of suggestions to return. Default is 3.

    Returns:
        List[str]: `n` decoded suggestions (stochastic; may vary run-to-run).
    """

    prompt = f"Business: {biz_desc}\nDomain suggestions:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    model.eval()
    with torch.inference_mode():
      outputs = model.generate(
          **inputs,
          max_new_tokens=32,
          num_return_sequences=n,
          temperature=0.7,
          do_sample=True,
      )
    prompt_len = inputs["input_ids"].shape[1]
    return [
        tokenizer.decode(o[prompt_len:], skip_special_tokens=True).strip()
        for o in outputs
    ]

## Zero shot function

In [16]:
# Zero shot function
def zero_shot_suggest(model, tokenizer, biz_desc, num_return_sequences=3):
    """
    Zero-shot domain suggestions from a business description.

    Builds a constrained prompt:
        "Business: {biz_desc}\nGive me exactly {N} domain names only, separated by commas, no other text.\nDomains:"
    Generates `N` sampled continuations and post-processes the text after the
    "Domains:" marker to extract domain-like strings via regex.

    Args:
        model: A Hugging Face causal LM (e.g., `AutoModelForCausalLM` or `PeftModel`).
        tokenizer: Matching tokenizer for `model`.
        biz_desc (str): Short description of the business.
        num_return_sequences (int, optional): Number of domain names to return. Default: 3.

    Returns:
        List[str]: Up to `num_return_sequences` domains parsed from the model output.

    """

    prompt = (
    f"Business: {biz_desc}\n"
    f"Give me exactly {num_return_sequences} domain names only, separated by commas, no other text.\n"
    "Domains:"
)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    model.eval()
    with torch.inference_mode():
      outputs = model.generate(
          **inputs,
          max_new_tokens=32,
          num_return_sequences=num_return_sequences,
          temperature=0.7,
          do_sample=True,
      )
    suggestions = []
    for o in outputs:
        text = tokenizer.decode(o, skip_special_tokens=True)
        # grab only what comes after our “Domains:” marker
        after = text.split("Domains:")[-1]
        # split by commas or newlines, strip whitespace/punctuation
        candidates = re.split(r"[,\n]+", after)
        for cand in candidates:
            print("cand: ",cand)
            c = cand.strip().strip(".-–* ")  # clean leading bullets/punctuation
            print("c: ",c)
            # keep only things that look like a domain
            if re.match(r"^[A-Za-z0-9][A-Za-z0-9\-_]+\.[A-Za-z]{2,}$", c):
                suggestions.append(c)
                print("True")
        # once we have 3, stop
        if len(suggestions) >= num_return_sequences:
            break
    return suggestions[:num_return_sequences]

## Evaluate BLEU and PPL

In [17]:
#Function to evaluate on the test set using bleu score
def Evaluate_bleu_n_perplexity(model, tokenizer, test_dataset):
  """
  Evaluate a causal LM on a tokenized test set using corpus BLEU and perplexity.

  Args:
      model: A Hugging Face causal LM (e.g., `AutoModelForCausalLM`/`PeftModel`).
      tokenizer: Matching HF tokenizer.
      test_dataset: A `datasets.Dataset` containing columns
          `["input_ids", "attention_mask", "labels"]`. `labels` may contain `-100`
          to mark ignored tokens.

  Returns:
      Tuple[float, float]: `(corpus_bleu, perplexity)`.

  """

  bleu_scores = []
  all_refs = []  # for corpus BLEU
  all_hyps = []
  if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
  # Keep model config in sync
  model.config.eos_token_id = tokenizer.eos_token_id
  model.config.pad_token_id = tokenizer.pad_token_id
  device = next(model.parameters()).device

  test_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
    device=torch.device(device)
  )
  model.eval()
  nll_sum = 0.0
  tok_count = 0
  with torch.inference_mode():
    for ex in test_dataset:
        # turn pre-tokenized lists into a 1×seq_len batch
        ids   = torch.tensor(ex["input_ids"]).unsqueeze(0)
        mask  = torch.tensor(ex["attention_mask"]).unsqueeze(0)
        labels = ex["labels"]
        labels_b = labels.unsqueeze(0)

        # Perplexity
        out = model(input_ids=ids,
                        attention_mask=mask,
                        labels=labels_b)  # HF computes CE mean over non-ignored tokens
        fill_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

        # count target tokens (ignore -100 if present)
        if (labels == -100).any():
            num_toks = (labels != -100).sum().item()
            labels_for_decode = labels.clone()
            labels_for_decode[labels_for_decode == -100] = fill_id
        else:
            pad = fill_id
            num_toks = (labels != pad).sum().item() if pad is not None else labels.numel()
            labels_for_decode = labels
        nll_sum += out.loss.item() * max(1, num_toks)
        tok_count += max(1, num_toks)
        #Bleu
        # generate top-1
        out_ids = model.generate(
            input_ids=ids,
            attention_mask=mask,
            #max_length=128,
            max_new_tokens=50,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            #early_stopping=True,
        )[0]


        if (labels == -100).any():
            labels = labels.clone()
            labels[labels == -100] = tokenizer.eos_token_id

        # decode
        ref_str = tokenizer.decode(labels, skip_special_tokens=True) #decode the reference
        pred = tokenizer.decode(out_ids, skip_special_tokens=True)

        # compute BLEU against the single reference
        ref = ref_str.split()
        hyp = pred.split()
        bleu_scores.append(sentence_bleu([ref], hyp))
        # collect for corpus BLEU
        all_refs.append([ref])   # note: list-of-list for possible multiple refs
        all_hyps.append(hyp)

  # Report
  avg_bleu = sum(bleu_scores) / len(bleu_scores)
  corpus_bleu_score = corpus_bleu(all_refs, all_hyps)
  ppl = math.exp(nll_sum / max(1, tok_count))
  #print(f"Average BLEU over {len(bleu_scores)} examples: {avg_bleu:.4f}")
  print(f"Corpus BLEU: {corpus_bleu_score:.4f}")
  print(f"Perplexity: {ppl:.4f}")
  return corpus_bleu_score, ppl


## The avg and std of all scores function

In [18]:
def calculate_avg_std_scores(model, tokenizer, nb_repetition, dataset):
  """
  Run repeated evaluations and report mean/std for BLEU, perplexity, and an LLM score.

  For `nb_repetition` passes over `dataset`, calls:
    - `Evaluate_bleu_n_perplexity(model, tokenizer, dataset)` → (bleu, ppl)
    - `Evaluate_llm_score_on_dataset(model, tokenizer, dataset)` → llm
  Aggregates the results, prints summary lines, and returns the averages and
  sample standard deviations.

  Args:
      model: Hugging Face causal LM (or compatible) to evaluate.
      tokenizer: Matching tokenizer for `model`.
      nb_repetition (int): Number of evaluation runs to average over.
      dataset: Tokenized dataset compatible with the evaluation functions.

  Returns:
      Tuple[float, float, float, float, float, float]:
          (avg_bleu, avg_ppl, avg_llm, std_bleu, std_ppl, std_llm)
  """

  bleu_score = []
  ppl_score = []
  llm_score = []
  for i in range(nb_repetition):
    bleu, ppl =  Evaluate_bleu_n_perplexity(model, tokenizer, dataset)
    llm = Evaluate_llm_score_on_dataset(model, tokenizer, dataset)
    bleu_score.append(bleu)
    ppl_score.append(ppl)
    llm_score.append(llm)
  avg_bleu = stats.mean(bleu_score)
  avg_ppl = stats.mean(ppl_score)
  avg_llm = stats.mean(llm_score)
  std_bleu = stats.stdev(bleu_score)
  std_ppl = stats.stdev(ppl_score)
  std_llm = stats.stdev(llm_score)
  print(f"The average: bleu score {avg_bleu}, perplexity score {avg_ppl}, and llm score {avg_llm}.")
  print(f"The std: bleu score {std_bleu}, perplexity score {std_ppl}, and llm score {std_llm}.")
  return avg_bleu, avg_ppl, avg_llm, std_bleu, std_ppl, std_llm

#Data Augmentation

In [19]:
# We will augment data by using synonyms of the words
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

def get_synonyms(word, max_syn=5):
    """Return up to max_syn WordNet synonyms for a given word (nouns only)."""
    synsets = wn.synsets(word, pos=wn.NOUN)
    lemmas = set(
        lemma.name().replace('_', '-').lower()
        for syn in synsets for lemma in syn.lemmas()
        if lemma.name().lower() != word.lower()
    )
    # limit to the most common ones
    return list(lemmas)[:max_syn]



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
def rule_variants(desc, domain):
    """
    Create domain variants by synonym-substituting each token in the base label.

    Args:
        desc (str): Arbitrary description carried alongside each variant.
        domain (str): Domain string; may have multi-part TLDs (e.g., "co.uk").

    Returns:
        list[tuple[str, str]]: Pairs of `(desc, "variant-domain.tld")`.
    """

    base, tld = domain.lower().split('.', 1)
    tokens = base.replace('-', ' ').split()
    out = []
    for i, tok in enumerate(tokens):
        for syn in get_synonyms(tok):
            new_tokens = tokens.copy()
            new_tokens[i] = syn
            slug = '-'.join(new_tokens)
            out.append((desc, f"{slug}.{tld}"))
    return out


In [21]:
# Apply to every row and collect
def augment_data(initial_data_csv, aug_path_csv):
  """
  Augment a (description, domain) CSV by adding rule-based domain variants.

  Reads `initial_data_csv` (expects columns: "business_description", "domain"),
  then for each row:
    - Keeps the original pair.
    - Extends with variants from `rule_variants(desc, dom)`.

  Args:
      initial_data_csv (str | pathlib.Path): Path to the input CSV.
      aug_path_csv (str | pathlib.Path): Intended output path for the augmented CSV
          (currently unused in this function).

  Returns:
      None

  """

  df = pd.read_csv(initial_data_csv)
  augmented = []
  for _, row in df.iterrows():
      desc = row["business_description"]
      dom  = row["domain"]
      # keep the original
      augmented.append({"business_description": desc, "domain": dom})
      # add rule-based variants
      for d, new_dom in rule_variants(desc, dom):
          augmented.append({"business_description": d, "domain": new_dom})

  # Build a DataFrame
  aug_df = pd.DataFrame(augmented).drop_duplicates()
  print(f"Original size: {len(df)}, Augmented size: {len(aug_df)}")
  aug_df.to_csv(aug_path_csv, index=False)
  return aug_df

#Safety check function

In [22]:
import requests

GROQ_API_KEY = getpass("Enter your Groq key:")
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"

def is_safe_with_groq(text):
    """
    Classify a text's safety using Groq's chat completion API.

    Args:
        text (str): The input text to evaluate.

    Returns:
        bool: `True` if the model replies "safe", else `False`.
    """

    prompt = f"""
    You are a content safety evaluator.

    Your task is to decide if the following text is unsafe due to containing:
    - sexually explicit content
    - hate speech or racism
    - graphic violence
    - illegal activity (e.g., drugs, weapons, gambling)
    - harassment or abuse

    Respond with one word only: "safe" or "unsafe".

    Text: "{text}"
    """

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "llama3-70b-8192",
        "messages": [
            {"role": "system", "content": "You are a content safety classifier."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0
    }

    try:
        response = requests.post(GROQ_URL, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()["choices"][0]["message"]["content"].strip().lower()
        return result == "safe"
    except Exception as e:
        print("Moderation error:", e)
        print("Response:", response.text)
        return False


Enter your Groq key:··········


#LLM as a Judge

## Judge

In [23]:
#GROQ_API_KEY = "your-groq-api-key"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"

def score_domain2(domain, business_description):
    """
    Score a domain name for a given business using Groq's chat completion API.

    Args:
        domain (str): The domain to evaluate (e.g., "acme.ai").
        business_description (str): Short description of the business/context.

    Returns:
        dict: A dictionary with keys:
            - "domain" (str): Echo of the input domain.
            - "score" (float): Normalized score in [0, 1] as produced by the model
              (prompt asks for sum/30 rounded to 2 decimals).
            - "confidence" (float): Model’s self-reported confidence in [0, 1].
          On any error or unparsable output, returns:
            {"domain": domain, "score": 0.0, "confidence": 0.0}
    """

    prompt = f"""
You are a brand expert evaluating domain names for businesses.

Task: score the domain for the business and ALSO report how confident you are in the score you assigned (not a formula).

Here is the business description:
"{business_description}"

Evaluate the following domain name: "{domain}"

Score it based on:
1. Memorability
2. Pronounceability
3. Brevity
4. Brandability
5. Relevance to the business description
6. Avoids ambiguity

Each is rated from 1 (Poor) to 5 (Excellent).
Then:
- total_score = sum of the six criterion scores (integer 6–30).
- score = <score/30, rounded to 2 decimals>
- confidence = your self-rated certainty in the total_score on a 0–1 scale with two decimals, where:
  0.90–1.00 = extremely confident (clear, unambiguous, strong fit)
  0.60–0.89 = moderately confident
  0.30–0.59 = low confidence (ambiguous/weak fit)
  0.00–0.29 = very low (insufficient info or highly ambiguous)

Return only this JSON format:
{{
  "domain": "{domain}",
  "score": <score>,
  "confidence": <float 0-1 with two decimals>
}}
"""

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "llama3-70b-8192",
        "messages": [
            {"role": "system", "content": "You are a domain evaluation expert."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.2
    }

    try:
        response = requests.post(GROQ_URL, headers=headers, json=payload)
        response.raise_for_status()

        content = response.json()["choices"][0]["message"]["content"]

        # Extract the first valid JSON block
        match = re.search(r"\{.*?\}", content, re.DOTALL)
        if match:
            json_block = match.group()
            result = json.loads(json_block)
            return {
                "domain": result["domain"],
                "score": result["score"],
                "confidence": result["confidence"]
            }
        else:
            raise ValueError("No JSON block found")

    except Exception as e:
        print("Error scoring domain:", e)
        print("Response:", response.text)
        return {"domain": domain, "score": 0.0, "confidence": 0.0}


## Final Generation function: safety check + generation + LLM scoring

In [24]:
def evaluate_business(business_description, llm_suggestor, tokenizer, nb_suggestions, zeroshot=False):
    """
    Generate and score domain suggestions for a business description with safety gating.

    Workflow:
      1) Safety check: calls `is_safe_with_groq(business_description)` and blocks
        the request if unsafe.
      2) Suggestion generation:
          - If `zeroshot=False`: uses `suggest_domains(llm_suggestor, tokenizer, business_description, nb_suggestions)`.
          - If `zeroshot=True` : uses `zero_shot_suggest(llm_suggestor, tokenizer, business_description, nb_suggestions)`.
      3) Scoring: for each suggested domain, calls `score_domain2(domain, business_description)`
        to obtain a JSON-like dict with `domain`, `score`, and `confidence`.
      4) Returns a result dict and prints it (or a blocked/error payload) as pretty JSON.

    Args:
        business_description (str): Short description of the business.
        llm_suggestor: Hugging Face text generation model used to propose domains.
        tokenizer: Matching tokenizer for `llm_suggestor`.
        nb_suggestions (int): Number of domain ideas to request.
        zeroshot (bool, optional): If True, use `zero_shot_suggest`; else `suggest_domains`.

    Returns:
        dict: One of:
            - {"suggestions": [...], "status": "success"}
              where each suggestion is the dict returned by `score_domain2`.
            - {"suggestions": [], "status": "blocked", "message": "..."} if safety fails.
            - {"suggestions": [], "status": "error", "message": "Domain generation failed"} if no domains.
    """

    #if not is_safe(business_description):
    if not is_safe_with_groq(business_description):
        blocked = {"suggestions": [],
            "status": "blocked",
            "message": "Request contains inappropriate content"}
        print(json.dumps(blocked, indent=2))
        return blocked

    # Generate domains
    if not zeroshot:
      domains = suggest_domains(llm_suggestor, tokenizer, business_description, nb_suggestions)
    else:
      domains = zero_shot_suggest(llm_suggestor, tokenizer, business_description, nb_suggestions)

    # Make sure domains are generated
    if not domains:
        return {
            "suggestions": [],
            "status": "error",
            "message": "Domain generation failed"
        }

    suggestions = [score_domain2(d,business_description) for d in domains]
    result = {
        "suggestions": suggestions,
        "status": "success"
    }
    print(json.dumps(result, indent=2))
    return result


## LLM evaluation on the test set

In [25]:
import time, random, re, statistics as stats
import torch
from requests import HTTPError

def _parse_retry_ms(msg: str, default_ms=500):
    m = re.search(r"try again in\s+(\d+)ms", msg)
    return int(m.group(1)) if m else default_ms

def safe_score_domain2(hyp_text, ref_text, max_retries=6, base_sleep=0.25):
    """Wrapper around score_domain2 that handles 429s and transient errors."""
    for attempt in range(max_retries):
        try:
            res = score_domain2(hyp_text, ref_text)  # <-- your existing scorer
            # expected shape: {"score": float, "confidence": float, ...}
            return res.get("score"), res.get("confidence"), None
        except HTTPError as e:
            status = getattr(e.response, "status_code", None)
            if status == 429:
                # Use server hint if present; otherwise exponential backoff with jitter
                try:
                    msg = e.response.json().get("error", {}).get("message", "")
                except Exception:
                    msg = str(e)
                wait_ms = _parse_retry_ms(msg, default_ms=int(1000 * (base_sleep * (2 ** attempt))))
                time.sleep(wait_ms / 1000.0 + random.uniform(0, 0.1))
                continue
            return None, None, f"HTTP {status or 'ERR'}: {e}"
        except Exception as e:
            return None, None, f"ERR: {e}"
    return None, None, "rate_limit_exceeded_after_retries"

In [26]:
#Function to evaluate on the test set using bleu score
def Evaluate_llm_score_on_dataset(model, tokenizer, test_dataset):
  """
  Evaluate a dataset by generating outputs and scoring them with an LLM-based scorer.

  Args:
      model: A Hugging Face causal LM (e.g., `AutoModelForCausalLM`/`PeftModel`).
      tokenizer: Matching tokenizer. If `pad_token` is missing, it is set to `eos_token`.
      test_dataset: A `datasets.Dataset` with columns `["input_ids", "attention_mask", "labels"]`.

  Returns:
      float: The mean LLM score over the dataset.

  """

  scores = []

  if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

  device = next(model.parameters()).device

  test_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
    device=torch.device(device)
  )
  model.eval()
  with torch.inference_mode():
    for ex in test_dataset:
        # turn pre-tokenized lists into a 1×seq_len batch
        ids   = torch.tensor(ex["input_ids"]).unsqueeze(0)
        mask  = torch.tensor(ex["attention_mask"]).unsqueeze(0)
        labels = ex["labels"]

        out_ids = model.generate(
            input_ids=ids,
            attention_mask=mask,
            max_new_tokens=50,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
        )[0]


        if (labels == -100).any():
            labels = labels.clone()
            labels[labels == -100] = tokenizer.pad_token_id

        # decode
        ref_ip = tokenizer.decode(ex["input_ids"], skip_special_tokens=True)
        pred = tokenizer.decode(out_ids, skip_special_tokens=True)
        ref = ref_ip.split()
        hyp = pred.split()
        #scores.append(score_domain2(hyp, ref)['score'])
        score, conf, err = safe_score_domain2(hyp, ref)
        if err is not None or score is None:
            failures += 1
            # small throttle to avoid bursts
            time.sleep(0.2)
            continue

        scores.append(score)

  # Report
  avg_score = stats.mean(scores)
  print(f"The average LLM score: {avg_score}.")
  return avg_score


##Logging

In [30]:
!pip install pynvml

In [27]:
import pynvml
# init NVML once
pynvml.nvmlInit()
_handle = pynvml.nvmlDeviceGetHandleByIndex(0)

class PerfLoggingCallback(TrainerCallback):

    def __init__(self, prefix="train/"):
        super().__init__()
        # placeholders for this step’s stats
        self.prefix = prefix
        self._step_start   = None
        self._step_time    = None
        self._grad_norm    = None
        self._gpu_mem_used = None
        self._gpu_util_pct = None

    def on_step_begin(self, args, state, control, **kwargs):
        # record the time just before forward()
        self._step_start = time.perf_counter()

    def on_step_end(self, args, state, control, **kwargs):
        # 1) step time
        self._step_time = time.perf_counter() - self._step_start

        # 2) gradient norm (retrieve model from kwargs)
        model = kwargs["model"]
        total_norm = 0.0
        for p in model.parameters():
            if p.grad is not None:
                total_norm += p.grad.detach().data.norm(2).item() ** 2
        self._grad_norm = total_norm ** 0.5

        # 3) GPU stats
        mem  = pynvml.nvmlDeviceGetMemoryInfo(_handle)
        util = pynvml.nvmlDeviceGetUtilizationRates(_handle)
        self._gpu_mem_used = mem.used / 1024**2        # MiB
        self._gpu_util_pct = util.gpu                  # percent

    def on_log(self, args, state, control, logs=None, **kwargs):
        """
        called whenever Trainer.flushes its logs (e.g. every logging_steps)
        `logs` already contains {'loss': ..., 'learning_rate': ..., etc.}
        """
        if logs is None:
            print("No logging")
            return control
        # only attach to training logs (they always have 'loss')
        if "loss" in logs:
            logs[self.prefix + "step_time"]    = self._step_time
            logs[self.prefix + "grad_norm"]    = self._grad_norm
            logs[self.prefix + "gpu_mem_used"] = self._gpu_mem_used
            logs[self.prefix + "gpu_util_pct"] = self._gpu_util_pct
        return control


#Training

In [28]:
clear_gpu_cache()
BASE_MODEL = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
#max_length = get_max_length(model)
max_length = 128 #Using this bc of memory limitation
print("Using max length of: ",max_length)
# get the df data
# Call the data generation function
data_csv = "synthetic_data.csv"
aug_data_csv = "aug_synthetic_data.csv"
df = generate_synthetic_data(data_csv)
print(f"Generated {len(df)} examples ")
dataset = preprocess_dataset(tokenizer, max_length, seed, data_csv)
# Augment the data
aug_df = augment_data(data_csv, aug_data_csv)
aug_dataset = preprocess_dataset(tokenizer, max_length, seed, aug_data_csv)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Using max length of:  128
Generated 2940 examples 


Generating train split: 0 examples [00:00, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/2381 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1128 [00:00<?, ? examples/s]

Map:   0%|          | 0/9135 [00:00<?, ? examples/s]

Map:   0%|          | 0/1015 [00:00<?, ? examples/s]

Map:   0%|          | 0/1128 [00:00<?, ? examples/s]

Length of training data: 9135, evaluation data: 1015, and test data: 1128


In [ ]:
%%time
torch.cuda.empty_cache()
# Train the model using LoRA on the inital dataset
print("Training model with LoRA on the initial data...")
_, lora_model = train_lora(dataset, tokenizer, output_dir="lora", base_model=BASE_MODEL, max_epochs=25)
# Merge the LoRA updates into the base weights, then unload the adapter module:
merged = lora_model.merge_and_unload()
# Save the merged model for future ease:
merged.save_pretrained("/content/drive/MyDrive/FamilyWall/models/lora_merged_initial")
merged.save_pretrained("lora_initial_merged")

Training model with LoRA on the initial data...


/tmp/ipython-input-1355666694.py:102: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainable parameter percentage: 0.14%


Epoch,Training Loss,Validation Loss
1,7.154100,4.658155
2,2.771300,1.339145
3,1.127300,1.050959
4,0.988100,0.940685
5,0.873500,0.820588
6,0.761400,0.732830
7,0.687900,0.663919
8,0.625400,0.603710
9,0.572100,0.544332
10,0.512100,0.494416


Saving to: lora_2025-08-08_21-40-40
CPU times: user 20min 42s, sys: 11min 8s, total: 31min 51s
Wall time: 32min 18s


In [ ]:
%%time
torch.cuda.empty_cache()
clear_gpu_cache()
# Hyperparameter optimization using optuna on the inital dataset
print("Searching for the optimal hyperparameters on the initial data...")
output_dir = "hpo"
best_trial, trainer = hyperparameter_search(dataset, tokenizer, output_dir, base_model_dir=BASE_MODEL, n_trials=10)
print("Best parameters: ", best_trial)

Searching for the optimal hyperparameters on the initial data...


/tmp/ipython-input-1761699797.py:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Using default LoRA params...


[I 2025-08-10 18:47:06,345] A new study created in memory with name: no-name-4bb2d514-2b3d-4175-aedc-c5e299d8459c


Using hyperparameter trial...


...

Epoch,Training Loss,Validation Loss
1,8.604000,7.603873
2,5.642400,2.400866
3,1.496500,1.052950
4,0.980400,0.906161
5,0.826800,0.745942
6,0.690900,0.648289
7,0.605100,0.568899
8,0.531200,0.499952
9,0.474300,0.455740
10,0.437700,0.426921


Early stopping at step 90 (epoch 18.00). Best 'eval_loss' = 0.3755248486995697 at hpo/run-9/checkpoint-65.


[I 2025-08-10 22:23:03,743] Trial 9 pruned. 


Best parameters:  BestRun(run_id='4', objective=0.28239572048187256, hyperparameters={'learning_rate': 0.00014660870013355224, 'num_train_epochs': 26, 'weight_decay': 0.02906463205437606, 'warmup_ratio': 0.2640957286449609, 'use_checkpointing': True, 'lora_r': 35, 'lora_alpha': 62, 'lora_dropout': 0.053726399998618175}, run_summary=None)


NameError: name 'PeftConfig' is not defined

In [ ]:
best_params = best_trial.hyperparameters
trial_dir = Path(trainer.args.output_dir) / f"run-{best_trial.run_id}"
print("Best model saved at: ",trial_dir)
optim_model = load_model_from_run(trial_dir)
optim_model.to(device)
optim_model.config.pad_token_id = optim_model.config.eos_token_id

Best model saved at:  hpo/run-4


In [ ]:
#Save the model
#train_metrics = train_out.metrics
eval_metrics = trainer.evaluate()
path = f"{output_dir}_best-run-{best_trial.run_id}"
drive_path = f"/content/drive/MyDrive/FamilyWall/models/{output_dir}_best-run-{best_trial.run_id}"
print("Saving to:", path)
trainer.save_model(path)
trainer.save_model(drive_path)
write_model_version(trainer, path, version="1.0.0", seed=42, dataset=dataset, eval_metrics=eval_metrics)
# merged = optim_model.merge_and_unload()
# # Save the merged model for future ease:
# merged.save_pretrained("/content/drive/MyDrive/FamilyWall/models/optim_merged_initial")
# merged.save_pretrained("optim_merged_initial")
torch.cuda.empty_cache()
clear_gpu_cache()

Saving to: hpo_best-run-4


**Training on the augmented test set**



1.   Using LoRA



In [ ]:
%%time
torch.cuda.empty_cache()
clear_gpu_cache()
#Redo using the augmented dataset
print("Redo the training on the augmented dataset")
# Train the model using LoRA on the augmented dataset
print("Training model with LoRA on the augmented data...")
_, lora_model_aug = train_lora(aug_dataset, tokenizer, output_dir="aug_lora", base_model=BASE_MODEL, max_epochs=25)
torch.cuda.empty_cache()
clear_gpu_cache()

Redo the training on the augmented dataset
Training model with LoRA on the augmented data...


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

/tmp/ipython-input-3312297701.py:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainable parameter percentage: 0.14%


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,3.302100,1.034184
2,0.802900,0.659017
3,0.544800,0.495234
4,0.451400,0.432664
5,0.398200,0.384006
6,0.358000,0.347773
7,0.327000,0.321079
8,0.304200,0.301703
9,0.288100,0.288529
10,0.276700,0.278680


Early stopping at step 576 (epoch 16.00). Best 'loss' = 0.24679209291934967 at aug_lora_2025-08-11_10-02-51/checkpoint-576.


Saving to: aug_lora_2025-08-11_10-02-51
CPU times: user 21min, sys: 2min 49s, total: 23min 50s
Wall time: 23min 45s




2.   Using HPO (not doing this bc of time and resources constraints)



In [ ]:
# %%time
# # Hyperparameter optimization using optuna on the augmented dataset
# print("Searching for the optimal hyperparameters on the augmented data...")
# best_trial_aug = hyperparameter_search(aug_dataset, tokenizer, base_model_dir=BASE_MODEL, n_trials=1)
# print("Best parameters: ", best_trial_aug)
# best_params_aug = best_trial_aug.hyperparameters
# # Train a model using the best hyperparameters on the augmented dataset
# torch.cuda.empty_cache()
# clear_gpu_cache()
# print("Training using the best parameters on the augmenetd data...")
# _, optim_model_aug = train_lora(aug_dataset, tokenizer, output_dir="aug_optimized", base_model = BASE_MODEL, r=best_params_aug["lora_r"], alpha=best_params_aug["lora_alpha"],dropout=best_params_aug["lora_dropout"],lr=best_params_aug["learning_rate"], max_epochs=1, warm=best_params_aug["warmup_ratio"], decay=best_params_aug["weight_decay"])
# # Merge the LoRA updates into the base weights, then unload the adapter module:
# merged = optim_model_aug.merge_and_unload()
# # Save the merged model for future ease:
# merged.save_pretrained("/content/drive/MyDrive/FamilyWall/models/hpo_merged_aug")
# merged.save_pretrained("optim_merged_aug")
# torch.cuda.empty_cache()
# clear_gpu_cache()

## Train with qwen

In [32]:
%%time
torch.cuda.empty_cache()
# Train the model using LoRA on the inital dataset
print("Training using qwen model with LoRA on the initial data...")
_, qwen_model = train_qwen_lora(dataset, tokenizer, max_epochs=10 )
# Optional: Merge the LoRA updates into the base weights, then unload the adapter module:
# merged = lora_model.merge_and_unload()
# # Save the merged model for future ease:
# merged.save_pretrained("/content/drive/MyDrive/FamilyWall/models/lora_merged_initial")
# merged.save_pretrained("lora_initial_merged")

Training using qwen model with LoRA on the initial data...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 40,370,176 || all params: 7,655,986,688 || trainable%: 0.5273
Trainable parameter percentage: 0.92%


/tmp/ipython-input-501555337.py:185: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,8.127300,5.715188
2,4.091100,3.026633
3,2.374900,2.126152
4,1.766600,1.710181
5,1.380800,1.482378
6,1.149400,1.344151
7,1.002300,1.291315
8,0.888000,1.242438
9,0.812400,1.229191
10,0.760700,1.220222


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Early stopping at step 380 (epoch 10.00). Best 'loss' = 1.2202221155166626 at qwen_2025-08-12_13-26-36/checkpoint-380.


Saving to: qwen_2025-08-12_13-26-36
CPU times: user 1h 29min 44s, sys: 24min 30s, total: 1h 54min 15s
Wall time: 1h 54min 15s


#Testing

##Test on the test dataset using the models trained on the initial train set

In [ ]:
# Load the trained model
# Load the model trained with lora
adapter_dir = "lora_2025-08-08_21-40-40"
cfg = PeftConfig.from_pretrained(adapter_dir)
base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
lora_model = PeftModel.from_pretrained(base, adapter_dir)
# Load the model traing with hpo
adapter_dir_hpo = "hpo/run-4/checkpoint-115"
cfg = PeftConfig.from_pretrained(adapter_dir_hpo)
base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
optim_model = PeftModel.from_pretrained(base, adapter_dir_hpo)

###PPL and BLEU scores

In [ ]:
# Evalute on the test set using lora model
lora_bleu, lora_ppl =  Evaluate_bleu_n_perplexity(lora_model, tokenizer, dataset["test"])
# Evalute on the test set using hpo model
hpo_bleu, hpo_ppl =  Evaluate_bleu_n_perplexity(optim_model, tokenizer, dataset["test"])
print(f"The average scores using LoRA: bleu {lora_bleu}, ppl {lora_ppl}.")
print(f"The average scores using HPO: bleu {hpo_bleu}, ppl {hpo_ppl}.")

/tmp/ipython-input-164270183.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ids   = torch.tensor(ex["input_ids"]).unsqueeze(0)
/tmp/ipython-input-164270183.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask  = torch.tensor(ex["attention_mask"]).unsqueeze(0)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end

Corpus BLEU: 0.8671
Perplexity: 1.3444
The average scores using LoRA: bleu 0.757765465823738, ppl 1.504977993504248.
The average scores using HPO: bleu 0.8670624232896204, ppl 1.3443991171516643.


###LLM score

In [ ]:
lora_llm = Evaluate_llm_score_on_dataset(lora_model, tokenizer, dataset["test"])
hpo_llm = Evaluate_llm_score_on_dataset(optim_model, tokenizer, dataset["test"])
print("The average LLM score using LoRA: ", lora_llm)
print("The average LLM score using HPO: ", hpo_llm)

/tmp/ipython-input-4179839472.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ids   = torch.tensor(ex["input_ids"]).unsqueeze(0)
/tmp/ipython-input-4179839472.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask  = torch.tensor(ex["attention_mask"]).unsqueeze(0)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-e

The average LLM score: 0.6531972789115646.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Error scoring domain: 503 Server Error: Service Unavailable for url: https://api.groq.com/openai/v1/chat/completions
Response: {"error":{"message":"Service unavailable. Visit https://groqstatus.com/ to see if there is an active incident.","type":"internal_server_error"}}



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Error scoring domain: 503 Server Error: Service Unavailable for url: https://api.groq.com/openai/v1/chat/completions
Response: {"error":{"message":"Service unavailable. Visit https://groqstatus.com/ to see if there is an active incident.","type":"internal_server_error"}}



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error scoring domain: 503 Server Error: Service Unavailable for url: https://api.groq.com/openai/v1/chat/completions
Response: {"error":{"message":"Service unavailable. Visit https://groqstatus.com/ to see if there is an active incident.","type":"internal_server_error"}}



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

The average LLM score: 0.731938775510204.
The average LLM score using LoRA:  0.6531972789115646
The average LLM score using HPO:  0.731938775510204


In [ ]:
torch.cuda.empty_cache()
clear_gpu_cache()

In [ ]:
print(f"Average scores using hpo model: bleu {lora_bleu}, perplexity {lora_ppl}, llm {lora_llm}.")
print(f"Average scores using hpo model: bleu {hpo_bleu}, perplexity {hpo_ppl}, llm {hpo_llm}.")

Average scores using hpo model: bleu 0.757765465823738, perplexity 1.504977993504248, llm 0.6531972789115646.
Average scores using hpo model: bleu 0.8670624232896204, perplexity 1.3443991171516643, llm 0.731938775510204.


##Test on the test dataset using the models trained on the augmented train set

In [26]:
# Load the trained model
# Load the model trained with lora
adapter_dir_aug = "aug_lora_2025-08-11_10-02-51"
cfg = PeftConfig.from_pretrained(adapter_dir_aug)
base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
lora_model_aug = PeftModel.from_pretrained(base, adapter_dir_aug)

###PPL and BLEU scores

In [ ]:
%%time
# Evalute on the test set using lora model
aug_bleu, aug_ppl =  Evaluate_bleu_n_perplexity(lora_model_aug, tokenizer, aug_dataset["test"])
print(f"The average scores using LoRA on the augmented dataset: bleu {aug_bleu}, ppl {aug_ppl}.")

/tmp/ipython-input-164270183.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ids   = torch.tensor(ex["input_ids"]).unsqueeze(0)
/tmp/ipython-input-164270183.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask  = torch.tensor(ex["attention_mask"]).unsqueeze(0)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end

Corpus BLEU: 0.7444
Perplexity: 1.2857
The average scores using LoRA on the augmented dataset: bleu 0.744377674606417, ppl 1.2857094776494788.
CPU times: user 6min 39s, sys: 5.58 s, total: 6min 45s
Wall time: 6min 45s


### LLM score

In [39]:
aug_llm = Evaluate_llm_score_on_dataset(lora_model_aug, tokenizer, aug_dataset["test"])
print("The average LLM score using augmeneted LoRA: ", aug_llm)

/tmp/ipython-input-1832953416.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ids   = torch.tensor(ex["input_ids"]).unsqueeze(0)
/tmp/ipython-input-1832953416.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask  = torch.tensor(ex["attention_mask"]).unsqueeze(0)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-e

Error scoring domain: 503 Server Error: Service Unavailable for url: https://api.groq.com/openai/v1/chat/completions
Response: {"error":{"message":"Service unavailable. Visit https://groqstatus.com/ to see if there is an active incident.","type":"internal_server_error"}}



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error scoring domain: 503 Server Error: Service Unavailable for url: https://api.groq.com/openai/v1/chat/completions
Response: {"error":{"message":"Service unavailable. Visit https://groqstatus.com/ to see if there is an active incident.","type":"internal_server_error"}}



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Error scoring domain: 503 Server Error: Service Unavailable for url: https://api.groq.com/openai/v1/chat/completions
Response: {"error":{"message":"Service unavailable. Visit https://groqstatus.com/ to see if there is an active incident.","type":"internal_server_error"}}



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Error scoring domain: Expecting ',' delimiter: line 2 column 221 (char 222)
Response: {"id":"chatcmpl-243513da-17d0-4fba-9a11-9fcc57d570b9","object":"chat.completion","created":1755000225,"model":"llama3-70b-8192","choices":[{"index":0,"message":{"role":"assistant","content":"{\n  \"domain\": \"['Business:', 'A', 'adventure', 'tour', 'operator', 'in', 'Tokyo.', 'Domain', 'suggestions:', 'adventure-tour-operator-yeddo.vacations', 'The', '2022', 'World', 'Cup:', 'What', 'does', 'it', 'mean', 'for', \"Japan's\", 'tourism', 'industry?']\",\n  \"score\": 0.13,\n  \"confidence\": 0.10\n}\n\nNote: I scored this domain very low because it's not a valid domain name, it's a sentence with multiple unrelated keywords. It's not memorable, pronounceable, or brief. It's also not relevant to the business description and is ambiguous."},"logprobs":null,"finish_reason":"stop"}],"usage":{"queue_time":0.071268974,"prompt_tokens":506,"prompt_time":0.015869239,"completion_tokens":161,"completion_time":0.552

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

The average LLM score: 0.6587765957446808.
The average LLM score using augmeneted LoRA:  0.6587765957446808


## Test using Qwen model

### PPL and BLEU scores

In [ ]:
# Load the qwen model
adapter_dir_qwen = "qwen_2025-08-12_13-26-36"
cfg = PeftConfig.from_pretrained(adapter_dir_aug)
base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
qwen_model = PeftModel.from_pretrained(base, adapter_dir_qwen)

In [33]:
%%time
# Evalute on the test set using lora model
qwen_bleu, qwen_ppl =  Evaluate_bleu_n_perplexity(qwen_model, tokenizer, dataset["test"])
print(f"The average scores using Lqwen model on the initial dataset: bleu {qwen_bleu}, ppl {qwen_ppl}.")

/tmp/ipython-input-2535150328.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ids   = torch.tensor(ex["input_ids"]).unsqueeze(0)
/tmp/ipython-input-2535150328.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask  = torch.tensor(ex["attention_mask"]).unsqueeze(0)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-e

Corpus BLEU: 0.6893
Perplexity: 213416.0005
The average scores using Lqwen model on the initial dataset: bleu 0.6893410648305851, ppl 213416.00050994847.
CPU times: user 33min 7s, sys: 5.7 s, total: 33min 13s
Wall time: 33min 13s


### LLM score

In [34]:
qwen_llm = Evaluate_llm_score_on_dataset(qwen_model, tokenizer, dataset["test"])
print("The average LLM score using qwen model: ", qwen_llm)

/tmp/ipython-input-2083117949.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ids   = torch.tensor(ex["input_ids"]).unsqueeze(0)
/tmp/ipython-input-2083117949.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask  = torch.tensor(ex["attention_mask"]).unsqueeze(0)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Error scoring domain: Expecting property name enclosed in double quotes: line 2 column 300 (char 301)
Response: {"id":"chatcmpl-053f35f8-5129-44e7-badc-ad180cc2b46d","object":"chat.completion","created":1755014073,"model":"llama3-70b-8192","choices":[{"index":0,"message":{"role":"assistant","content":"{\n  \"domain\": \"['Business:', 'A', 'vintage', 'apparel', 'store', 'in', 'Paris.', 'Domain', 'suggestions:', 'vintageapparelstoreparis.net', 'personalized-new', \\\"york.solutions.toursydney.money-istanbul.mediayhat'ost.boutiqueistanoronto.store.store.propertyokyo.com.b-rome.net.farm.video-london.com-\", 'porque.bhat-beirut.homes,’t']\",\n  \"score\": 0.07,\n  \"confidence\": 0.10\n}\n\nNote: This domain name is a jumbled collection of words and phrases, making it extremely difficult to evaluate. It appears to be a combination of multiple domain name suggestions, which makes it hard to assess its relevance to the business description. The score is very low due to the domain's lack of co

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for

Error scoring domain: Expecting ',' delimiter: line 2 column 379 (char 380)
Response: {"id":"chatcmpl-ee0d27be-65ac-4052-bc43-30df3c361bab","object":"chat.completion","created":1755014205,"model":"llama3-70b-8192","choices":[{"index":0,"message":{"role":"assistant","content":"{\n  \"domain\": \"['Business:', 'A', 'flight', 'price', 'prediction', 'engine', 'in', 'London,', 'specializing', 'in', 'dynamic', 'pricing', 'engines.', 'Domain', 'suggestions:', 'journeyFlightlondon.tours', 'migrations.tv', 'mascaratoulouse.shop-beirut.coffee198.store.studioingsydney.com.bomeision-new', 'york.earthlos', 'angelos.property-be-tokyo.com', 'Detroit.h_H-toronto.come(\"\u003c?-losrok', 'protagonists.shop.h']\",\n  \"score\": 0.07,\n  \"confidence\": 0.10\n}\n\nNote: This domain name is a collection of multiple domain names, which is not a typical scenario. I'll evaluate it as a single domain name, but please note that it's not a valid or practical domain name.\n\nHere's the breakdown of the scores:\n\

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

Error scoring domain: Unterminated string starting at: line 2 column 13 (char 14)
Response: {"id":"chatcmpl-5e7a29ef-21d0-4340-95a5-b22a16c131fe","object":"chat.completion","created":1755016270,"model":"llama3-70b-8192","choices":[{"index":0,"message":{"role":"assistant","content":"Here is the evaluation of the domain name:\n\n{\n  \"domain\": \"['Business:', 'A', 'crop', 'yield', 'predictor', 'in', 'Tokyo.', 'Domain', 'suggestions:', 'cropyieldpredictortokyo.com', 'crop.financeingsystemrome.organic.studio-london.stistanbul.solutionsydney.sttoulouse.s’t', 'ih.agyWC.stille', 'control.agtok', '})', '.shop.aiubai.travel', 'than.st', 'interaction-ly', 'shallow.io.ag.stpl']\",\n  \"score\": 0.13,\n  \"confidence\": 0.40\n}\n\nHere's the breakdown of the scores:\n\n1. Memorability: 1 (Poor) - The domain name is extremely long and contains multiple unrelated words, making it difficult to remember.\n2. Pronounceability: 1 (Poor) - The domain name is a mix of words and abbreviations, making it 

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Error scoring domain: Expecting ',' delimiter: line 2 column 371 (char 372)
Response: {"id":"chatcmpl-82f6b632-7902-4d2c-9c4f-7fe46ddfe2bf","object":"chat.completion","created":1755016311,"model":"llama3-70b-8192","choices":[{"index":0,"message":{"role":"assistant","content":"{\n  \"domain\": \"['Business:', 'A', 'language', 'school', 'in', 'Toronto,', 'specializing', 'in', 'real-time', 'progress', 'tracking.', 'Domain', 'suggestions:', 'language-school-toronto.academy', 'personalizedew', 'york.earthsar.organic(w.com.b.media.homes.solutions', '=\u003ewiseydney.com.engineeringystem.hbeforetokyo.com.agency-d’tob.ecoiances.money.ai.android.property,’t=\".b\\'’ty']\",\n  \"score\": 0.07,\n  \"confidence\": 0.10\n}\n\nNote: This domain name is extremely poor and does not make sense. It appears to be a jumbled collection of words and phrases, and it's difficult to discern any relevance to the business description."},"logprobs":null,"finish_reason":"stop"}],"usage":{"queue_time":0.089957576,"

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


The average LLM score: 0.2975510204081633.
The average LLM score using qwen model:  0.2975510204081633


## Multiple runs for improvement analysis

In [ ]:
torch.cuda.empty_cache()
clear_gpu_cache()
print("Evaluatin on the test set, using LoRA model.")
lora_bleu_avg, lora_ppl_avg, lora_llm_avg, lora_bleu_std, lora_ppl_std, lora_llm_std = calculate_avg_std_scores(lora_model, tokenizer, 5, dataset["test"])
torch.cuda.empty_cache()
clear_gpu_cache()
print("Evaluatin on the test set, using HPO model.")
hpo_bleu_avg, hpo_ppl_avg, hpo_llm_avg, hpo_bleu_std, hpo_ppl_std, hpo_llm_std = calculate_avg_std_scores(optim_model, tokenizer, 5, dataset["test"])
torch.cuda.empty_cache()
clear_gpu_cache()
print("Evaluatin on the augmented test set.")
aug_bleu_avg, aug_ppl_avg, aug_llm_avg, aug_bleu_std, aug_ppl_std, aug_llm_std = calculate_avg_std_scores(lora_model_aug, tokenizer, 5, aug_dataset["test"])
torch.cuda.empty_cache()
clear_gpu_cache()

In [ ]:
print(f"The average scores using LoRA model on initial dataset:")
print(f"BLEU: {lora_bleu_avg}, PPL: {lora_ppl_avg}, LLM: {lora_llm_avg}.")
print(f"The std scores using LoRA model on initial dataset:")
print(f"BLEU: {lora_bleu_std}, PPL: {lora_ppl_std}, LLM: {lora_llm_std}.")
print("-"*50)
print(f"The average scores using HPO model on initial dataset:")
print(f"BLEU: {hpo_bleu_avg}, PPL: {hpo_ppl_avg}, LLM: {hpo_llm_avg}.")
print(f"The std scores using LoRA model on initial dataset:")
print(f"BLEU: {hpo_bleu_std}, PPL: {hpo_ppl_std}, LLM: {hpo_llm_std}.")
print('-'*50)
print(f"The average scores using LoRA model on augmented dataset:")
print(f"BLEU: {aug_bleu_avg}, PPL: {aug_ppl_avg}, LLM: {aug_llm_avg}.")
print(f"The std scores using LoRA model on augmented dataset:")
print(f"BLEU: {aug_bleu_std}, PPL: {aug_ppl_std}, LLM: {aug_llm_std}.")

# Checking tensorboards

## LoRA initial dataset

In [ ]:
#run this cell to check the tensorboard of LoRA + initial dataset model
%load_ext tensorboard
%tensorboard --logdir lora_2025-08-08_21-40-40

## HPO initial dataset

In [ ]:
#run this cell to check the tensorboard of HPO + initial dataset model
%load_ext tensorboard
%tensorboard --logdir hpo/runs

## LoRA augmented dataset

In [46]:
#run this cell to check the tensorboard of LoRA + augmented dataset model
%load_ext tensorboard
%tensorboard --logdir aug_lora_2025-08-11_10-02-51

## Qwen initial dataset

In [48]:
#run this cell to check the tensorboard of Qwen + initial dataset model
%load_ext tensorboard
%tensorboard --logdir qwen_2025-08-12_13-26-36

# Edge Case discovery

In [45]:
import re
import csv
from collections import Counter
from datetime import datetime

# Load the trained model
# Load the model trained with lora
adapter_dir = "lora_2025-08-08_21-40-40"
cfg = PeftConfig.from_pretrained(adapter_dir)
base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
lora_model = PeftModel.from_pretrained(base, adapter_dir)
# Load the model traing with hpo
adapter_dir_hpo = "hpo/run-4/checkpoint-115"
cfg = PeftConfig.from_pretrained(adapter_dir_hpo)
base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
optim_model = PeftModel.from_pretrained(base, adapter_dir_hpo)

BASE_MODEL = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Load the model trained with lora on aug dataset
adapter_dir_aug = "aug_lora_2025-08-11_10-02-51"
cfg = PeftConfig.from_pretrained(adapter_dir_aug)
base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
lora_model_aug = PeftModel.from_pretrained(base, adapter_dir)

# Load the qwen model
# adapter_dir_qwen = "qwen_2025-08-12_13-26-36"
# cfg = PeftConfig.from_pretrained(adapter_dir_aug)
# base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
# qwen_model_aug = PeftModel.from_pretrained(base, adapter_dir_qwen)

# Seed edge-case prompts
EDGE_PROMPTS = [
    "Domain for an app 🚀 that tracks underwater penguins",
    "Funny yet serious legal advice platform",
    "Dark horror theme for preschool education",
    "AI-powered cola beverage brand similar to a famous soda (avoid trademarks)",
    "Domain that looks like a popular payment site using homoglyphs",
    "Ultra short name for a fintech app",
    "Global interplanetary cryptocurrency, eco-friendly, supports 500 languages",
    "Domain with emoji-only brand 😺",
    "Luxury funeral services in Japan",
]

# Simple rules
DOMAIN_REGEX = re.compile(
    r"^(?=.{1,253}$)(?:[a-z0-9](?:[a-z0-9-]{0,61}[a-z0-9])?\.)+[a-z]{2,63}$"
)
EMOJI_PATTERN = re.compile(r"[\U00010000-\U0010ffff]", flags=re.UNICODE)

TRADEMARK_LIKE = {"coca", "cola", "google", "apple", "microsoft", "nike", "paypal", "amazon", "meta", "facebook", "instagram", "disney"}
PHISHING_LIKE = {"paypai", "paypaI", "googIe", "appIe"}  # common homoglyph tricks
INAPPROPRIATE = {"xxx", "porn", "nsfw"}  # keep coarse/clean
CONTRADICTIONS = [
    ("funny", "serious"),
    ("horror", "preschool"),
    ("edgy", "corporate"),
]

def classify(prompt: str, domain: str):
    labels = []
    reasons = []

    dl = (domain or "").strip()
    pl = (prompt or "").lower()

    # Format checks
    if not dl:
        labels.append("Format Error"); reasons.append("Empty output")
        return labels, "; ".join(reasons)

    if EMOJI_PATTERN.search(dl):
        labels.append("Format Error"); reasons.append("Emoji in domain")

    if not DOMAIN_REGEX.match(dl.lower()):
        labels.append("Format Error"); reasons.append("Fails domain regex")

    if len(dl) > 253:
        labels.append("Format Error"); reasons.append("Domain too long")

    for part in dl.split("."):
        if not part:
            labels.append("Format Error"); reasons.append("Empty label")
        if len(part) > 63:
            labels.append("Format Error"); reasons.append("Label >63 chars")
        if part.startswith("-") or part.endswith("-"):
            labels.append("Format Error"); reasons.append("Label starts/ends with '-'")

    # Sensitivity / legal heuristics (coarse)
    low = dl.lower()
    if any(w in low for w in INAPPROPRIATE):
        labels.append("Inappropriate Content"); reasons.append("Adult-ish keyword")

    if any(w in low for w in TRADEMARK_LIKE):
        labels.append("Legal/Trademark Risk"); reasons.append("Trademark-like string")

    if any(w in low for w in PHISHING_LIKE):
        labels.append("Phishing/Impersonation Risk"); reasons.append("Homoglyph/similarity")

    # Prompt contradictions
    for a, b in CONTRADICTIONS:
        if a in pl and b in pl:
            labels.append("Semantic Contradiction"); reasons.append(f"Prompt asks for {a} and {b}")
            break

    # Dedup & join
    labels = list(dict.fromkeys(labels))
    reason = "; ".join(dict.fromkeys(reasons)) or "OK"
    return labels or ["None"], reason

def main():
    rows = []
    label_counter = Counter()
    total = 0
    failed = 0

    for prompt in EDGE_PROMPTS:
        total += 1
        try:
            domain = evaluate_business(prompt, qwen_model, tokenizer, 1)['suggestions'][0]['domain']
        except Exception as e:
            domain = ""
            labels, reason = ["Runtime Error"], f"Exception: {e}"
        else:
            labels, reason = classify(prompt, domain)

        if labels != ["None"]:
            failed += 1
            for L in labels:
                label_counter[L] += 1

        rows.append({
            "prompt": prompt,
            "generated_domain": domain,
            "labels": " | ".join(labels),
            "reason": reason
        })

    # Write CSV
    with open("results_qwen.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["prompt","generated_domain","labels","reason"])
        writer.writeheader()
        writer.writerows(rows)

    # Write summary
    with open("summary_qwen.txt", "w", encoding="utf-8") as f:
        f.write(f"Edge Check Summary ({datetime.utcnow().isoformat()}Z)\n")
        f.write("===========================================\n")
        f.write(f"Total prompts: {total}\n")
        f.write(f"Any-failure outputs: {failed} ({(failed/total*100):.1f}%)\n\n")
        f.write("Counts by label:\n")
        if label_counter:
            for label, cnt in label_counter.most_common():
                f.write(f"- {label}: {cnt}\n")
        else:
            f.write("- None (no failures detected)\n")

    print("Done. See results_hpo.csv and summary_hpo.txt")

if __name__ == "__main__":
    main()


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


{
  "suggestions": [
    {
      "domain": "engagingbeirut.invest-sydney.com(Locationnav-los angelos.property.fit-toronto.store.property,olutions=\"losightlondon.earth.\"",
      "score": 0.07,
      "confidence": 0.1
    }
  ],
  "status": "success"
}


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


{
  "suggestions": [
    {
      "domain": "globe",
      "score": 0.43,
      "confidence": 0.6
    }
  ],
  "status": "success"
}


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

{
  "suggestions": [
    {
      "domain": "eco-friendlynew york.organicc.financeenokyo.com(detail.net.ai. Domain suggestions: eco-friendlynew york.organicc.financeenokyo",
      "score": 0.13,
      "confidence": 0.29
    }
  ],
  "status": "success"
}


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


{
  "suggestions": [
    {
      "domain": "wander-Mertoronto.organic-toronto eg.shop-beirut.homes,olutions =>.industry, and emphasizes community focus. Domain suggestions: wander-re",
      "score": 0.13,
      "confidence": 0.2
    }
  ],
  "status": "success"
}


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


{
  "suggestions": [
    {
      "domain": "adventure.g-poursydney.com wanted.marketing-sydney.industry, and emphasizes community focus. Domain suggestions: adventure\u201dc compleydney.net\u0646\u06af",
      "score": 0.13,
      "confidence": 0.2
    }
  ],
  "status": "success"
}
Done. See results_hpo.csv and summary_hpo.txt


# Testing the model

In [43]:
# This is an example of how to use the model for domain name suggestion
# I will test using the examples provided in the assigment

# First we chould choose what model to use, I will use the optimized model

# Load the model traing with hpo
adapter_dir_hpo = "hpo/run-4/checkpoint-115"
cfg = PeftConfig.from_pretrained(adapter_dir_hpo)
base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
optim_model = PeftModel.from_pretrained(base, adapter_dir_hpo)

business_description = "organic coffee shop in downtown area"
result = evaluate_business(business_description, optim_model, tokenizer, 3) # 3 states the nb of suggestions to return
print("#"*50)
business_description = "adult content website with explicit nude content"
result = evaluate_business(business_description, optim_model, tokenizer, 3) # 3 states the nb of suggestions to return

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{
  "suggestions": [
    {
      "domain": "organic-coffee-shop-downtown.com",
      "score": 0.83,
      "confidence": 0.8
    },
    {
      "domain": "organic-coffeshop-downtown.net",
      "score": 0.83,
      "confidence": 0.8
    },
    {
      "domain": "organic-coffeshop-downtown.com",
      "score": 0.83,
      "confidence": 0.8
    }
  ],
  "status": "success"
}
##################################################
{
  "suggestions": [],
  "status": "blocked",
  "message": "Request contains inappropriate content"
}


In [ ]:
# To change the selected model, ucomment the desired model
# fill in the desired description and run to see the output

# For HPO model
# Load the model traing with hpo
adapter_dir_hpo = "hpo/run-4/checkpoint-115"
cfg = PeftConfig.from_pretrained(adapter_dir_hpo)
base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
model = PeftModel.from_pretrained(base, adapter_dir_hpo)

# Load the LoRA model on inital dataset
# adapter_dir = "lora_2025-08-08_21-40-40"
# cfg = PeftConfig.from_pretrained(adapter_dir)
# base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
# model = PeftModel.from_pretrained(base, adapter_dir)

# Load The LoRA model on augmented dataset
# adapter_dir_aug = "aug_lora_2025-08-11_10-02-51"
# cfg = PeftConfig.from_pretrained(adapter_dir_aug)
# base = AutoModelForCausalLM.from_pretrained(cfg.base_model_name_or_path, low_cpu_mem_usage=True)
# model = PeftModel.from_pretrained(base, adapter_dir_aug)


# Enter your description here
business_description = ""
result = evaluate_business(business_description, model, tokenizer, 3) # 3 states the nb of suggestions to return, change it as you like